In [3]:
from gensim.models import KeyedVectors
filename = '/home/adithya/glove.6B/glovew2v.6B.100d.txt'
model = KeyedVectors.load_word2vec_format(filename, binary=False)

In [10]:
result = model.most_similar(positive=['evaporate'], negative=['water','sun'], topn=1)
print(result)

[('recomendados', 0.6044833064079285)]


In [46]:
print(model.similarity('evaporate','sun'))
print(model.most_similar('imbibe'))

0.14574410646849545
[('quaff', 0.5747246146202087), ('flaunt', 0.5675700902938843), ('internalize', 0.5549746751785278), ('excretes', 0.5462298393249512), ('assimilate', 0.5260298252105713), ('overeat', 0.5237395763397217), ('metabolize', 0.5203923583030701), ('sublimate', 0.5196496844291687), ('overanalyze', 0.5181325674057007), ('partake', 0.5151299238204956)]


In [12]:
from nltk.corpus import wordnet as wn

def morphify(word):

    """ morph a word """
    #synsets = wn.synsets(word, pos=org_pos)
    synsets = wn.synsets(word)

    # Word not found
    if not synsets:
        return []

    # Get all  lemmas of the word
    #lemmas = [l for s in synsets for l in s.lemmas() if s.name().split('.')[1] == org_pos]
    lemmas = [l for s in synsets for l in s.lemmas()]

    # Get related forms
    derivationally_related_forms = [(l, l.derivationally_related_forms()) for l in lemmas]

    # filter only the targeted pos
    #related_lemmas = [l for drf in derivationally_related_forms for l in drf[1] if l.synset().name().split('.')[1] == target_pos]
    related_lemmas = [l for drf in derivationally_related_forms for l in drf[1]]
    related_lemmas2 = [drf[0] for drf in derivationally_related_forms]

    # Extract the words from the lemmas
    #words = [l.name() for l in related_lemmas]
    words = []
    for l in related_lemmas:
        words.append(l.synset().name().split('.')[0])
        words.append(l.name())
    for l in related_lemmas2:
        words.append(l.synset().name().split('.')[0])
        words.append(l.name())

    return set(words)

#print(morphify('runs'))

In [13]:
#a = ['string','pitch','short']
b = morphify('high')
print(b)
dictio = {}
for c in b:
    if '_' in c:
        c = c.split('_')[0]
    try :
        z = 0
        for d in a:
            x = model.similarity(d,c)
            z += x
        #dictio[c] = z
        dictio[c] = (z / len(a))
    except:
        print(c,' NOT PRESENT IN VOCAB')

z = sorted(dictio.items(), key=lambda x: x[1], reverse=True)
print(z)

{'mellow', 'heights', 'high_school', 'luxuriously', 'high-pitched', 'eminent', 'high_gear', 'gamey', 'eminence', 'in_high_spirits', 'gamy', 'senior_high', 'senior_high_school', 'highness', 'game', 'mellowness', 'richly', 'highschool', 'high', 'high_up'}
mellow  NOT PRESENT IN VOCAB
heights  NOT PRESENT IN VOCAB
high  NOT PRESENT IN VOCAB
luxuriously  NOT PRESENT IN VOCAB
high-pitched  NOT PRESENT IN VOCAB
eminent  NOT PRESENT IN VOCAB
high  NOT PRESENT IN VOCAB
gamey  NOT PRESENT IN VOCAB
eminence  NOT PRESENT IN VOCAB
in  NOT PRESENT IN VOCAB
gamy  NOT PRESENT IN VOCAB
senior  NOT PRESENT IN VOCAB
senior  NOT PRESENT IN VOCAB
highness  NOT PRESENT IN VOCAB
game  NOT PRESENT IN VOCAB
mellowness  NOT PRESENT IN VOCAB
richly  NOT PRESENT IN VOCAB
highschool  NOT PRESENT IN VOCAB
high  NOT PRESENT IN VOCAB
high  NOT PRESENT IN VOCAB
[]


In [40]:
from nltk.stem.porter import *
stemmer = PorterStemmer()

In [48]:
# keywords = ['high','string','pitch','short']
def query_expansion(keyword_list):
    finalwordslist = []
    for word in keyword_list:
        #print(word)
        a = keyword_list.copy()
        a.remove(word)
        #print(a)
        b = morphify(word)
        #print(b)
        dictio = {}
        for c in b:
            if '_' in c:
                c = c.split('_')[0]
            try :
                z = 0
                for d in a:
                    x = model.similarity(d,c)
                    z += x
                #dictio[c] = z
                dictio[c] = (z / len(a))
            except:
                continue

        z = sorted(dictio.items(), key=lambda x: x[1], reverse=True)
        #print(z[0:5])
        #print('\n')
        for eachword in z[0:5]:
            finalwordslist.append(eachword[0])
    wordslist = [stemmer.stem(plural) for plural in finalwordslist]
    return wordslist
keywords = ['Water', 'evaporates', 'sun', 'leaving', 'salt'] 
#query_expansion(keywords)

In [49]:
import spacy

#text = "Water evaporates in the sun, leaving behind salt"
nlp = spacy.load('en_core_web_sm')

In [50]:

def generate_similarword(text):
    doc = nlp(text)
    
    keywords_dict = {}
    keywords = []
    for token in doc:
        if 'NN' in token.tag_ or 'VB' in token.tag_:
            keywords_dict[token.text] = token.tag_
            keywords.append(token.text.lower())
        #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,token.shape_, token.is_alpha, token.is_stop)
    
    #print(keywords)
    similarwords = query_expansion(keywords)
    return similarwords

In [51]:
import pickle
ans_pairs = pickle.load(open("./pairs","rb"))    #reference answer and student answer
labels = pickle.load(open("./labels","rb"))      #correct or incorrect are the labels
print(len(labels))   # number of labels
print(len(ans_pairs)) #number of question pairs
print(len(list(set(labels))))

4969
4969
2


In [52]:
reference_list = []
student_list = []
for i in range(len(labels)):
    reference_list.append(generate_similarword(ans_pairs[i][0]))
    student_list.append(generate_similarword(ans_pairs[i][1]))

In [53]:
print(reference_list[0])

['take', 'draw', 'absorb', 'concentr', 'soak', 'heat', 'hot', 'high', 'fire', 'heat', 'full', 'energi', 'free', 'push', 'depart', 'clean', 'white', 'snow', 'egg', 'blank', 'think', 'reflect', 'shine', 'reflect', 'specul', 'heat', 'hot', 'high', 'fire', 'heat', 'full', 'energi', 'free', 'push', 'depart', 'cover', 'cover', 'cut', 'top', 'get', 'box', 'packag', 'packag', 'corner', 'pack', 'heat', 'hot', 'high', 'fire', 'heat', 'cover', 'cover', 'cut', 'top', 'get', 'box', 'packag', 'packag', 'corner', 'pack']


In [65]:
testlabels = []
for i in range(len(labels)):
    if(len(set(reference_list[i]) & set(student_list[i])) > 5):
        testlabels.append(1)
    else:
        testlabels.append(0)

In [66]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(labels,testlabels))
print(confusion_matrix(labels,testlabels))

             precision    recall  f1-score   support

          0       0.67      0.56      0.61      2961
          1       0.48      0.59      0.53      2008

avg / total       0.59      0.57      0.58      4969

[[1650 1311]
 [ 816 1192]]
